In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# NLTK
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import warnings
warnings.filterwarnings('ignore')

## Data Loading 

In [2]:
df = pd.read_json('./JEOPARDY_QUESTIONS1.json')

In [3]:
df.head()

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [4]:
questions = df['question']

## Preprocessing

### Tokenization

In [5]:
def tokenization(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

tokenized_sentences = list(tokenization(questions))

print(tokenized_sentences [1:3])

[['no', 'olympian', 'football', 'star', 'at', 'carlisle', 'indian', 'school', 'mlb', 'seasons', 'with', 'the', 'reds', 'giants', 'braves'], ['the', 'city', 'of', 'yuma', 'in', 'this', 'state', 'has', 'record', 'average', 'of', 'hours', 'of', 'sunshine', 'each', 'year']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenized_sentences, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_sentences], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[tokenized_sentences[0]]])

['for', 'the', 'last', 'years', 'of', 'his', 'life', 'galileo', 'was', 'under', 'house_arrest', 'for', 'espousing', 'this', 'man', 'theory']


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
tokenized_without_stopwords = remove_stopwords(tokenized_sentences)

# Form Bigrams
tokenized_without_stopwords_bigrams = make_bigrams(tokenized_without_stopwords)


nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
lemmatized_words = lemmatization(tokenized_without_stopwords_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [9]:
print(lemmatized_words[1:9])

[['olympian', 'season', 'red', 'giant', 'brave'], ['city', 'state', 'record', 'average', 'hour', 'sunshine', 'year'], ['live', 'art', 'linkletter', 'show', 'company', 'serve', 'billionth', 'burger'], ['second', 'president', 'united_state'], ['title', 'aesop_fable', 'insect', 'share', 'billing', 'grasshopper'], ['build', 'still', 'use', 'today'], ['steal', 'baron', 'steal', 'bull'], ['winter', 'record', 'inch', 'snow', 'fall', 'ranger', 'station', 'state']]


In [10]:
# Create Dictionary
dictionary = corpora.Dictionary(lemmatized_words)

# Create Corpus
texts = lemmatized_words

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [11]:
# Create Dictionary
dictionary = corpora.Dictionary(lemmatized_words)

# Create Corpus
texts = lemmatized_words

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,  id2word=dictionary,num_topics= 6)

In [13]:
doc_lda = lda_model[corpus]
for index, topics in lda_model.show_topics(formatted=False, num_words=50):
        print('Topic: {} \nWords: {}'.format(index+1,[w[0] for w in topics] ))


Topic: 1 
Words: ['river', 'state', 'die', 'type', 'president', 'year', 'serve', 'part', 'make', 'first', 'use', 'mile', 'long', 'man', 'water', 'say', 'get', 'body', 'back', 'start', 'live', 'actor', 'land', 'good', 'foot', 'become', 'go', 'form', 'line', 'study', 'car', 'tree', 'bird', 'take', 'order', 'look', 'eye', 'plant', 'begin', 'earth', 'run', 'house', 'ride', 'find', 'time', 'send', 'add', 'grow', 'food', 'letter']
Topic: 2 
Words: ['name', 'first', 'bear', 'title', 'write', 'last', 'woman', 'novel', 'play', 'book', 'film', 'man', 'become', 'know', 'make', 'set', 'author', 'call', 'little', 'character', 'work', 'begin', 'base', 'team', 'child', 'wear', 'black', 'classic', 'brother', 'well', 'also', 'red', 'blue', 'include', 'sell', 'famous', 'young', 'wife', 'family', 'use', 'join', 'late', 'story', 'publish', 'king', 'star', 'son', 'english', 'nickname', 'rule']
Topic: 3 
Words: ['name', 'word', 'play', 'mean', 'come', 'country', 'year', 'film', 'make', 'song', 'first', 'use

In [14]:
report="This chilled leek & potato soup is traditionally topped with chopped chives"
report = report.split(" ")

for index, proportion in lda_model[dictionary.doc2bow(report)]:
    print('The proportion of topic {} is {} percent'.format(index+1, proportion*100))


The proportion of topic 1 is 63.63515257835388 percent
The proportion of topic 2 is 3.2477542757987976 percent
The proportion of topic 3 is 3.2475531101226807 percent
The proportion of topic 4 is 3.247946873307228 percent
The proportion of topic 5 is 23.373456299304962 percent
The proportion of topic 6 is 3.2481368631124496 percent
